<a href="https://colab.research.google.com/github/AcydAce/FoDs/blob/main/FoDs_Final_Assignment_Steam_Recommender_Bayu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import re

import itertools
import matplotlib.pyplot as plt
from google.colab import files
import io

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

!pip install gradio
import gradio as gr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.8/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
dataset = files.upload()

Saving steam.csv to steam (2).csv


In [12]:
#checks dataset
df = pd.read_csv(io.BytesIO(dataset['steam.csv']))

print(df.shape)
df.head()

(27075, 18)


,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [13]:
df.columns

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price'],
      dtype='object')

In [14]:
# extracts years
def extract_year(date):
   year = date[:4]
   
   # some games do not have the info about year in the column title.
   if year.isnumeric():
      return int(year)
   else:
      return np.nan

df['year'] = df['release_date'].apply(extract_year)
df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,year
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,2000
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,1999
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,2003
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,2001
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,1999


In [15]:
# Creates score column
def create_score(row):
  pos_count = row['positive_ratings']  
  neg_count = row['negative_ratings']
  total_count = pos_count + neg_count
  average = pos_count / total_count
  return round(average, 2)

def total_ratings(row):
  pos_count = row['positive_ratings']  
  neg_count = row['negative_ratings']
  total_count = pos_count + neg_count
  return total_count

df['total_ratings'] = df.apply(total_ratings, axis=1)
df['score'] = df.apply(create_score, axis=1)


In [16]:
# Calculate mean of vote average column
m_sco = df['score'].mean()

# Calculates the minimum number of votes required to be in the chart which is req
req = df['total_ratings'].quantile(0.90)
print(req)

908.6000000000022


In [17]:
# calculate the weighted rating for each qualified game
# Function that computes the weighted rating of each game
def weighted_rating(x, req=req, m_sco=m_sco):
    rat = x['total_ratings']
    sco = x['score']
    
    return round((rat/(rat+req) * sco) + (req/(req+rat) * m_sco), 2)

# Define a new feature 'score' and calculate its value with `weighted_rating()`
df['weighted_score'] = df.apply(weighted_rating, axis=1)

#Prints the top 15 games
df[['name', 'total_ratings', 'score', 'weighted_score']].head(15)

,name,total_ratings,score,weighted_score
0,Counter-Strike,127873,0.97,0.97
1,Team Fortress Classic,3951,0.84,0.82
2,Day of Defeat,3814,0.90,0.86
3,Deathmatch Classic,1540,0.83,0.79
4,Half-Life: Opposing Force,5538,0.95,0.92
5,Ricochet,3442,0.80,0.78
6,Half-Life,28855,0.96,0.95
7,Counter-Strike: Condition Zero,13559,0.89,0.88
8,Half-Life: Blue Shift,4242,0.90,0.87
9,Half-Life 2,70321,0.97,0.97


In [18]:
# we need to connect by '-' before we split them by ' ' for tags with multiple words
df['steamspy_tags'] = df['steamspy_tags'].str.replace(' ','-')
#TFIDF
df['genres'] = df['steamspy_tags'].str.replace(';',' ')

# count the number of occurences for each genre in the data set
counts = dict()
for i in df.index:

  #for each element in list (each row, split by ' ', in genres column) 
  #-- we're splitting by space so tfidf can interpret the cells
   for g in df.loc[i,'genres'].split(' '):

#if element is not in counts(dictionary of genres)
      if g not in counts:

        #give genre dictonary entry the value of 1
         counts[g] = 1
      else:
        #increase genre dictionary entry by 1
         counts[g] = counts[g] + 1

#Test Genre Counts
counts.keys()
print(counts['Puzzle'])

1116


In [19]:
# creates an object for TfidfVectorizer
tfidf_vector = TfidfVectorizer(stop_words='english')

# convert the list of rows of genre tags into a matrix 
tfidf_matrix = tfidf_vector.fit_transform(df['genres'])

tfidf_matrix.shape
#(games[rows], genres[columns])

(27075, 370)

In [20]:
# creates the cosine similarity matrix
sim_matrix = linear_kernel(tfidf_matrix,tfidf_matrix) 
print(sim_matrix)

[[1.         1.         0.53785954 ... 0.16328518 0.         0.        ]
 [1.         1.         0.53785954 ... 0.16328518 0.         0.        ]
 [0.53785954 0.53785954 1.         ... 0.         0.         0.        ]
 ...
 [0.16328518 0.16328518 0.         ... 1.         0.61573349 0.61573349]
 [0.         0.         0.         ... 0.61573349 1.         1.        ]
 [0.         0.         0.         ... 0.61573349 1.         1.        ]]


In [21]:
# creates a function to find the closest title name
def matching_score(a,b):
  #fuzz.ratio(a,b) calculates the Levenshtein Distance between a and b, and returns the score for the distance
   return fuzz.ratio(a,b)

In [22]:
#These functions are needed to return different attributes of the recommended game titles

def get_title_year_from_index(index):
   return df[df.index == index]['year'].values[0]

def get_title_from_index(index):
   return df[df.index == index]['name'].values[0]

def get_index_from_title(title):
   return df[df.name == title].index.values[0]

def get_score_from_index(index):
   return df[df.index == index]['score'].values[0]

def get_weighted_score_from_index(index):
   return df[df.index == index]['weighted_score'].values[0]

def get_total_ratings_from_index(index):
   return df[df.index == index]['total_ratings'].values[0]

def get_platform_from_index(index):
  return df[df.index == index]['platforms'].values[0]

In [23]:
# A function to return the most similar title to the words a user type
def find_closest_title(title):

  #matching_score(a,b) > a is the current row, b is the title we're trying to match
   leven_scores = list(enumerate(df['name'].apply(matching_score, b=title))) 
   sorted_leven_scores = sorted(leven_scores, key=lambda x: x[1], reverse=True) 
   closest_title = get_title_from_index(sorted_leven_scores[0][0])
   distance_score = sorted_leven_scores[0][1]
   return closest_title, distance_score

   #returns the top 10 closest titles
def closest_names(title):
   leven_scores = list(enumerate(df['name'].apply(matching_score, b=title)))
   sorted_leven_scores = sorted(leven_scores, key=lambda x: x[1], reverse=True)
   top_closest_names = [get_title_from_index(i[0]) for i in sorted_leven_scores[:10]] 
   return top_closest_names

closest_names('siege')

['Besiege',
 'Deadsiege',
 'SiegeVR',
 'Zenge',
 'Ridge',
 'Krieg',
 'System Siege',
 'Inside Me',
 'Insincere',
 'Rising']

In [24]:
#uses gradio.app

def gradio_contents_based_recommender_v2(game, how_many, dropdown_option, sort_option, min_year, platform, min_score):
  #Return closest game title match
  closest_title, distance_score = find_closest_title(dropdown_option)
  #Create a Dataframe with these column headers
  recomm_df = pd.DataFrame(columns=['Game Title', 'Year', 'Score', 'Weighted Score', 'Total Ratings'])
  #Make the closest title whichever dropdown option the user has chosen
  closest_title = dropdown_option
  #find the corresponding index of the game title
  games_index = get_index_from_title(closest_title)
  #return a list of the most similar game indexes as a list
  games_list = list(enumerate(sim_matrix[int(games_index)]))
  #Sort list of similar games from top to bottom
  similar_games = list(filter(lambda x:x[0] != int(games_index), sorted(games_list,key=lambda x:x[1], reverse=True)))
  #Print the game title the similarity matrix is based on
  print('Here\'s the list of games similar to '+'\033[1m'+str(closest_title)+'\033[0m'+'.\n')
  #Only return the games that are on selected platform
  n_games = []
  for i,s in similar_games:
    if platform in get_platform_from_index(i):
      n_games.append((i,s))
  #Only return the games that are above the minimum score
  high_scores = []
  for i,s in n_games:
    if get_score_from_index(i) > min_score:
      high_scores.append((i,s))
    
  #Return the game tuple (game index, game distance score) and store in a dataframe
  for i,s in n_games[:how_many]: 
    #Dataframe will contain attributes based on game index
    row = {'Game Title': get_title_from_index(i), 'Year': get_title_year_from_index(i), 'Score': get_score_from_index(i), 
           'Weighted Score': get_weighted_score_from_index(i), 
           'Total Ratings': get_total_ratings_from_index(i),}
    #Append each row to this dataframe       
    recomm_df = recomm_df.append(row, ignore_index = True)
  #Sort dataframe by Sort_Option provided by user
  recomm_df = recomm_df.sort_values(sort_option, ascending=False)
  #Only include games released same or after minimum year selected
  recomm_df = recomm_df[recomm_df['Year'] >= min_year]

  return recomm_df

#Create list of unique calendar years based on main df column
years_sorted = sorted(list(df['year'].unique()))
#Ask user for input
print("What games do you want most similar to?:")

names = closest_names(input())
#Interface will include these buttons based on parameters in the function with a dataframe output
dropdown = gr.Interface(gradio_contents_based_recommender_v2, ["text", gr.inputs.Slider(1, 20, step=int(1)), 
                                                            gr.inputs.Dropdown(names), 
                                                            gr.inputs.Radio(['Year','Score','Weighted Score','Total Ratings']),
                                                            gr.inputs.Slider(int(years_sorted[0]), int(years_sorted[-1]), step=int(1)),
                                                            gr.inputs.Radio(['windows','xbox','playstation','linux','mac']),
                                                            gr.inputs.Slider(0, 10, step=0.1)],
                        "dataframe")

dropdown.launch(debug=True)

What games do you want most similar to?:
powerwash simulator


/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:88: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:216: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:182: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Here's the list of games similar to Vaporwave Simulator.

Here's the list of games similar to Vaporwave Simulator.

Keyboard interruption in main thread... closing server.
